In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
# from wordcloud import WordCloud
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors
from scipy import stats
from warnings import filterwarnings
from IPython.core.display import display, HTML
import requests
from urllib.request import urlopen
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
from pyquery import PyQuery 
import json
import sys
import asyncio
import aiohttp
from aiohttp import ClientSession, ClientConnectorError
import pathlib
import sys
import asyncio
import time 
import aiohttp
from aiohttp.client import ClientSession

display(HTML("<style>.container { width:80% !important; }</style>"))
filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set(font_scale=1.25)

In [2]:
books = pd.read_csv('../../../data/recommendation/Books.csv',index_col=0)
ratings = pd.read_csv('../../../data/recommendation/Ratings.csv')

print(books.shape)
books.head()

(271360, 7)


,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
ISBN,,,,,,,
0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
m=5
R = ratings[['ISBN', 'Book-Rating']].groupby(['ISBN'], as_index=True).mean()
C = ratings['Book-Rating'].mean()
ratings = pd.concat([R, ratings.groupby(['ISBN'], as_index=True).size().to_frame()],axis=1)
ratings['weighted_average'] = ratings.apply(lambda row: (row[0]/(row[0]+m))*row['Book-Rating'] + (m/(row[0]+m))*C, axis=1)

ratings = ratings.rename(columns={'0':'Votes'})
print(ratings.shape)
ratings.head()

(340556, 3)


,Book-Rating,0,weighted_average
ISBN,,,
0330299891,3.0,2,2.904964
0375404120,1.5,2,2.476393
0586045007,0.0,1,2.389125
9022906116,3.5,2,3.047822
9032803328,0.0,1,2.389125


In [4]:
index = [idx for idx in books.index if idx in ratings.index]
df = pd.concat([books.loc[index,:], ratings.loc[index, ['weighted_average']]], axis=1)
print(df.shape)
df.head()

(270151, 8)


,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,weighted_average
ISBN,,,,,,,,
0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.389125
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.386040
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,3.666844
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,3.833422
0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,2.389125


In [5]:
# %%time

# data = {}
# for isbn in tqdm(df.index):
#     url = 'https://openlibrary.org/books/OL3597106M/%s'%(isbn)
#     html = requests.get(url).text
#     soup = BeautifulSoup(html, 'html.parser')
#     res = [link.get('href') for link in soup.find_all('a') if link.get('href')!=None]
#     data[isbn] = list(set([i.replace('/subjects/','') for i in res if i.startswith(('/subjects/'))]))
    
#     json.dump(data, open('data.json', 'w'))

In [6]:
# import urllib.request
# from tornado import ioloop, httpclient
url_list = ['https://openlibrary.org/books/OL3597106M/%s'%(isbn) for isbn in df.index][:1000]

In [7]:
# import nest_asyncio
# nest_asyncio.apply()
# __import__('IPython').embed()

In [8]:
from urllib.parse import urlparse
from threading import Thread
import http.client, sys
from queue import Queue

In [9]:
%%time
concurrent = 200
data = {}

def doWork():
    while True:
        url = q.get()
        status, url = getStatus(url)
        doSomethingWithResult(status, url)
        q.task_done()

def getStatus(ourl):
    try:
        url = urlparse(ourl)
        conn = httplib.HTTPConnection(url.netloc)   
        conn.request("HEAD", url.path)
        res = conn.getresponse()
        return res.status, ourl
    except:
        return "error", ourl

def doSomethingWithResult(status, url):
    isbn = url.split('/')[-1]
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    res = [link.get('href') for link in soup.find_all('a') if link.get('href')!=None]
    data[isbn] = list(set([i.replace('/subjects/','') for i in res if i.startswith(('/subjects/'))]))
    
#     print(status, url)

q = Queue(concurrent * 2)
for i in range(concurrent):
    t = Thread(target=doWork)
    t.daemon = True
    t.start()
try:
    for url in tqdm(url_list):
        q.put(url.strip())
    q.join()
except KeyboardInterrupt:
    sys.exit(1)
json.dump(data, open('data.json', 'w'))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:32<00:00, 30.48it/s]


Wall time: 1min 17s


In [10]:
# async def fetch_html(url: str, session: ClientSession, **kwargs) -> tuple:
#     try:
#         resp = await session.request(method="GET", url=url, **kwargs)
#     except ClientConnectorError:
#         return (url, 404)
#     return (url, resp.status)

# async def make_requests(urls: set, **kwargs) -> None:
#     async with ClientSession() as session:
#         tasks = []
#         for url in urls:
#             tasks.append(fetch_html(url=url, session=session, **kwargs))
#         results = await asyncio.gather(*tasks)

#     for result in results:
#         print(result)
#         print(f'{result[1]} - {str(result[0])}')

# urls = ['https://openlibrary.org/books/OL3597106M/%s'%(isbn) for isbn in df.index][:2]  
# await make_requests(urls=urls)

In [11]:
sys.exit()

SystemExit: 

In [ ]:
# Action and Adventure
# Classics
# Comic Book or Graphic Novel
# Detective and Mystery
# Fantasy
# Historical Fiction
# Horror
# Literary Fiction
# Romance
# Science Fiction (Sci-Fi)
# Short Stories
# Suspense and Thrillers
# Women's Fiction
# Biographies and Autobiographies
# Cookbooks
# Essays
# History
# Memoir
# Poetry
# Self-Help
# True Crime


# Data visualization

In [ ]:
def clean_numeric(x):
    try:
        return float(x)
    except:
        return np.nan

def clean_string(x):
    try:
        return str(x)
    except:
        return np.nan

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
df['type'].value_counts().plot(kind='bar') # Bar plot of anime type

# Compute occurence of individual categories
list_genre_flat = [clean_string(i).split(', ') for i in list(df_filtered['genre'].copy())]
list_genre_flat = pd.Series([i for s in list_genre_flat for i in s]) # Flatenned list. Each occurence contribute to individual categories
plt.subplot(1,2,2)
list_genre_flat.value_counts().plot(kind='bar') # Bar plot of anime type

Comedy genre is the category which is the most present in this filtered dataset.

## Anime data correlation - TV show

Is there a numerical correlation betweenn user's rating, number of episode and number of members belonging to anime community.
We will focus on TV anime.


In [ ]:
df_filtered = df_filtered.loc[df_filtered['type']=='TV']

df_filtered['episodes'] = df_filtered['episodes'].apply(clean_numeric).astype('float') # Replace 0 value with NaN and clean
df_filtered['rating'] = df_filtered['rating'].apply(clean_numeric).astype('float') # Transform to proper numerical data
df_filtered['members'] = df_filtered['members'].apply(clean_numeric).astype('float') # Transform to proper numerical data

df_filtered['episodes'] = df_filtered['episodes'].fillna(df_filtered['episodes'].median())
df_filtered['rating'] = df_filtered['rating'].fillna(df_filtered['rating'].median())
df_filtered['members'] = df_filtered['members'].fillna(df_filtered['members'].median())
# Since we deleted row, it is important to reindex the dataframe, so we don't have missing index
df_filtered = df_filtered.reset_index()
list_index = df_filtered['index'].unique()

df_filtered.head(10)

In [ ]:
plt.figure(figsize=(20,7))
plt.subplot(1,3,1)
sns.distplot(df_filtered['episodes'])
plt.subplot(1,3,2)
sns.distplot(df_filtered['rating'])
plt.subplot(1,3,3)
sns.distplot(df_filtered['members'])

In [ ]:
sns.set(font_scale=1)
corr = df_filtered[['episodes','rating','members']].corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(9,9))

with sns.axes_style("white"):
    ax = sns.heatmap(corr, mask=mask, vmax=.3, square=True, annot=True)

# Data preparation
Set categorical data and their contribution the type TV (large number of episodes).

In [ ]:
def get_index(x):
    return df_filtered[df_filtered['name']==x].index.tolist()[0]
def recommend_me(anime):
    index = get_index(anime)
    print('Here are 10 anime similar to', anime, ' - ', df_filtered.loc[index]['genre'],'\n')
    rec = []
    for j in anime_indices[index][1:]:
        print(df_filtered.loc[j]['name'],' - ', df_filtered.loc[j]['genre'])
        rec.append([df_filtered.loc[j]['anime_id'], df_filtered.loc[j]['name'], df_filtered.loc[j]['genre']])
    rec = pd.DataFrame(rec)
#     rec = rec.transpose()
    rec.columns = ['id', 'name', 'genre']
    return rec

In [ ]:
df_rating = pd.read_csv('../data/anime-recommendations-database/rating.csv')
user_no = 1500

# Select data by number of users
df_rating = df_rating.head(df_rating[df_rating['user_id']==df_rating['user_id'].unique()[user_no]].index[-1])

# Removing anime which where removed earlier in the notebook (list of anime). Reset index for avoinding conflict
df_rating = df_rating[~df_rating['anime_id'].isin(list_index)].reset_index(drop=True)

# User rating data investigation
df_count = pd.DataFrame({'user_id': df_rating['user_id'].unique()})
df_count['watched'] = [df_rating[df_rating['user_id']==i].shape[0] for i in df_rating['user_id'].unique()]
df_count['vote_no'] = [len([j for j in list(df_rating[df_rating['user_id']==i]['rating']) if j != -1]) for i in df_rating['user_id'].unique()]
df_count['ratio'] = df_count['vote_no'] / df_count['watched']

# Establish 'like' ranges: 0 to 5 = Dislike; 5 to 7 = Neutral; over 7 = Like
df_count['like'] = [len([j for j in list(df_rating[df_rating['user_id']==i]['rating']) if j !='NaN' and j >= 7]) for i in df_rating['user_id'].unique()]
df_count['neutral'] = [len([j for j in list(df_rating[df_rating['user_id']==i]['rating']) if j !='NaN' and 5 <= j < 7]) for i in df_rating['user_id'].unique()]
df_count['dislike'] = [len([j for j in list(df_rating[df_rating['user_id']==i]['rating']) if j !='NaN' and 0 <= j < 5]) for i in df_rating['user_id'].unique()]
df_count['like_id'] = [[j[1] for j in list(zip(df_rating[df_rating['user_id']==i]['rating'],df_rating[df_rating['user_id']==i]['anime_id'])) if j[0] !='NaN' and j[0] >= 7] for i in df_rating['user_id'].unique()]

sns.set(font_scale=1.5)
plt.figure(figsize=(24,8))
plt.subplot(1,3,1)
sns.distplot(df_count['vote_no'])
plt.subplot(1,3,2)
sns.distplot(df_count['watched'])
plt.subplot(1,3,3)
sns.distplot((100*df_count['ratio']).astype(int))

# Investigate user common base - Collaborative filtering

Check one user like history, check similar user and reccomend based on common viewing history

In [ ]:
# Select users with more than 10 votes
df_user = df_count[df_count['vote_no']>=10].reset_index(drop=True)
df_user.head()

In [ ]:
# Select user cluster with common like history (at least 10 items common, 10 different items). We investigate user_id = 3
user_no = 5
set_user = np.array(df_user[df_user['user_id'] == user_no]['like_id'].to_numpy()[0])
df_user = df_user[df_user['user_id'] != user_no].reset_index(drop=True)

# Check number of comon items in anime_id list
df_user['common'] = [len([j for j in df_user[df_user['user_id'] == i]['like_id'].to_numpy()[0] if j in set_user]) for i in df_user['user_id'].tolist()]

# Remove items with less than 10 common items
df_user = df_user[df_user['common']>=10].reset_index(drop=True)
df_user['different'] = [len([j for j in df_user[df_user['user_id'] == i]['like_id'].to_numpy()[0] if j not in set_user]) for i in df_user['user_id'].tolist()]
df_user['unseen'] = [[j for j in df_user[df_user['user_id'] == i]['like_id'].to_numpy()[0] if j not in set_user] for i in df_user['user_id'].tolist()]

df_user.sort_values(by='common', ascending=False).head()

In [ ]:
# User 5 best ratings and corresponding genre
user = df_rating[df_rating['user_id']==user_no]
user = user[user['rating'] != -1].drop(['user_id'],axis=1).reset_index(drop=True)
user = user[['rating','anime_id']]
user['name'] = [str(df_filtered.loc[df_filtered['anime_id']==i]['name'].tolist())[2:-2] for i in user['anime_id']]
user['genre'] = [str(df_filtered.loc[df_filtered['anime_id']==i]['genre'].tolist())[2:-2] for i in user['anime_id']]
# user = user[user['user_id']==user_no]
user.sort_values(by='rating',ascending=False).head(10)

In [ ]:
# Count number of similar ID in unseen movies
top_common = pd.value_counts(pd.Series([i for s in df_user['unseen'].tolist() for i in s]))
test = pd.DataFrame(top_common)
test['id'] = test.index
test.columns = 'count','id'
test['name'] = [str(df_filtered.loc[df_filtered['anime_id']==i]['name'].tolist())[2:-2] for i in test['id']]
test['genre'] = [str(df_filtered.loc[df_filtered['anime_id']==i]['genre'].tolist())[2:-2] for i in test['id']]
test = test.reset_index(drop=True)
test.head()

# Item based recommendation

In [ ]:
genre_filtered = df_filtered['genre'].str.get_dummies(sep=',')
X = pd.concat([genre_filtered, df_filtered['rating'], df_filtered['members']],axis=1)
scaled = MaxAbsScaler()
X = scaled.fit_transform(X)
reg = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(X)
anime_indices = reg.kneighbors(X)[1] # picks off the array for anime indices
rec = recommend_me('Shingeki no Kyojin')
rec.head(10)

In [ ]:
# Add user similarity section - it also include dislike anime